# Music Generation using Algo Approch
I utilized a modified version of the 16x16 tone matrix concept to generate music. Specifically, I processed an image and used it as a switch to activate specific elements in the tone matrix.

## Import Dependencies
For image manipulation, OpenCV library has been utilized.
To process symbolic music representation, Music21 library has been implemented.
For carrying out array and matrix operations during the process, NumPy library has been used.
To display and save high-quality images in higher dimensions, Matplotlib library has been incorporated.

In [ ]:
import cv2
import numpy as np
import music21 as m21
import matplotlib.pyplot as plt

In [ ]:
SEQ_LEN = 32

In [ ]:
# Load the image
img = cv2.imread('Warren-Buffett-On-Rules.jpg')

# Resize the image to 16x16 pixels
resized_img = cv2.resize(img, (SEQ_LEN, 16))

# Convert the image to grayscale
gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

# for working with different color scale, split image into b, g, r separately
b, r, g = cv2.split(resized_img)
print(f"Shape of one space image: {b.shape}")

## Extracting intensity information column-wise
The algorithm extracts the intensity information from the image column-wise. For each column, it calculates either the maximum intensity or the minimum intensity value according to user choice.

In [ ]:
def get_intensity_columnwise(resized_img, seq_len=16, approach='max'):
    """
    Get the intensity values of the resized image.

    Args:
        resized_img: numpy array
            The resized image as a 2D array with only one color scale.
        seq_len: int, optional (default: 16)
            The length of the generated music sequence. The width of the image is divided into this many segments.
        approach: str, optional (default: 'max')
            The approach to use when mapping pixel intensity to musical notes. 'max' uses the maximum pixel intensity,
            while 'average' uses the average intensity.

    Returns:
        column_index: list
            A list of the indices of the maximum (or average) intensity value in each column of the image.
    """
    intensity_values = resized_img.flatten()
    print(f'Shape of intensity_values: {intensity_values.shape} and resized image_shape: {resized_img.shape}')

    # Transpose the intensity values to get the values column-wise
    intensity_values_column_wise = [intensity_values[i::seq_len] for i in range(seq_len)]

    column_index = []
    
    # Print the index of the intensity values column-wise
    for i, col in enumerate(intensity_values_column_wise):
        print(f'Index: {i} and Length of col: {len(col)}')
        if approach == 'max':
            indices = np.where(col == col.max())[0].tolist()
        elif approach == 'min':
            indices = np.where(col == col.min())[0].tolist()
        else:
            print(f"approch can be 'min' or 'max'. \nPlease Check your code where approach={approach} is invalid")
        column_index.append(indices)
        print(f'Column {i}: {col} and maxindex: {indices}')
    return column_index

In [ ]:
max_intensity_list = get_intensity_columnwise(b, seq_len=SEQ_LEN)
# min_intensity_list = get_intensity_columnwise(b, seq_len=16, approach='min')

In [ ]:
max_intensity_list

## Plot images with separate color space

In [ ]:
plt.figure(figsize=(16,SEQ_LEN))
images_color_space = (b, g, r)
image_label = ['Blue', 'Green', 'Red']
for index in range(3):
    plt.subplot(1, 3, index+1)
    plt.title(f"{image_label[index]} Color Space")
    plt.imshow(images_color_space[index], cmap='gray')
plt.show()

## Tone matrix Configuration
The Tone matrix is a 2D matrix where the horizontal axis represents time and the vertical axis represents frequency. As we move from bottom to top on the vertical axis, the frequency increases. Similarly, as we move from left to right on the horizontal axis, time increases. The matrix consists of 16 fixed notes, where each note is assigned a unique row ranging from the lower frequency note C3 to the upper frequency note C6.

In [ ]:
# make 16x16 tone matrix
switch = np.zeros((16, SEQ_LEN), dtype='object')
notes = ['C6', 'A5', 'G5', 'F5', 'D5', 'C5', 'A4', 'G4',
        'F4', 'D4', 'C4', 'A3', 'G3', 'F3', 'D3', 'C3']
for i in range(16):
  switch[i, :] = notes[i]
print(f"Tone matrix shape: {switch.shape}")

## Construct Switch configuration
If there are more switches that are to be opened in column-wise, then switch with least frequency is played. Here, column with highest index has lowest frequency

The pure black and white image is constructed by interchanging rows and columns for easier calculation. It has dimension Nx16. To get image with dimension equal to tone matrix, we transpose the array to get dimension 16xN. 

In [ ]:
def construct_switch_config(intensity, seq_len=16, config='s'):
    '''
    Constructs a black and white image based on the given intensity values and configuration.

    Args:
        intensity: np.array or list -> A list containing a list of intensity indices.
        seq_len: int -> The length of the time axis. Default value is 16.
        config: str -> Configuration for the switch matrix. Possible values are:
                's' (single): Only one note can be activated in a column at a time.
                'two': A maximum of two notes can be played at a time.
                'three': A maximum of three notes can be played at a time.
                'all': No limit to the number of notes that can be played at a time. (can be from 1 to 16)

    Result:
        bw_image: np.array -> A 2D array representing the black and white image where white represents the ON state 
                              and black represents the OFF state.
    '''
    bw_image = np.zeros((16, 16), dtype='uint8')
    if config.lower() == 's' or config.lower()=='single':
        for i, index_col in enumerate(intensity):
            col = index_col[-1]
            bw_image[i][col] = 255
            
    if config.lower()=='two':
        for i, index_col in enumerate(intensity):
            col = index_col[-1]
            bw_image[i][col] = 255
            try:
                col = index_col[-2]
                bw_image[i][col] 
            except IndexError:
                continue
        
    if config.lower == 'all':
        for i, index_col in enumerate(intensity):
            for col in index_col:
                bw_image[i][col] = 255
    bw_image = bw_image.T
    return bw_image
plt.figure(figsize=(16,16))
plt.imshow(bw_image, cmap='gray')
plt.show()

## Creating melody
The tone matrix is configured with a switch setting (black and white) to produce the melody.

In [ ]:
def construct_melody(image, tone_matrix, instrument='Piano', seq_len=16):
    """
    Generate a Music21 Stream object.

    Args:
        image: numpy array or list, shape (16, seq_len)
            The image that serves as the basis for the melody.
        tone_matrix: numpy array or list, shape (16, seq_len)
            The tone matrix that corresponds to the image.
        instrument: str, optional (default: 'Piano')
            The instrument to use when creating the melody.
        seq_len: int, optional (default: 16)
            The length of the melody, in number of notes.

    Returns:
        A Music21 Stream object representing the generated melody.
    """
    stream_algo = m21.stream.Stream()
    instrument_func = m21.instrument.fromString(instrument)
    stream_algo.insert(0.0, instrument_func)
    offset = 0
    for y in range(seq_len):
        add = ''
        for x in range(16):
            if (image[x][y] != 0):
                temp = tone_matrix[x][y]
                add = add + ' ' + temp
            stream_algo.insertIntoNoteOrChord(offset, m21.chord.Chord(add))
        if (len(add.split()) == 1):
            offset += 0.5
            print(f"{offset} offset has Chord: {add}")
        else: 
            offset += 1
    return stream_algo

### Save in midi format

In [ ]:
stream_algo = construct_melody(image=bw_image, tone_matrix=switch, seq_len=SEQ_LEN)
stream_algo.show('text')
stream_algo.write('midi', 'algo_one_note_q.mid')